In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
print("Okay")

Okay


In [2]:
import pandas as pd
df=pd.read_csv("TRAIN.csv")
df.head()

,Index,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,2.245295,-1.160960,-1.966682,-1.430190,-0.607246,-1.508696,-0.074415,-0.655096,-1.970141,...,0.010490,0.521714,-0.064971,0.048849,0.383290,0.103970,-0.047350,-0.064800,74.75,0
1,1,1.278609,0.102574,0.512079,0.003930,-0.723474,-1.334105,0.029958,-0.296595,0.128119,...,-0.081470,-0.182506,0.078986,0.789993,0.219794,0.938359,-0.078720,0.008119,1.38,0
2,2,1.466457,0.026088,-0.499298,-0.674372,-0.144883,-1.178075,0.058089,-0.420145,-1.359651,...,-0.234087,-0.710542,-0.001242,-0.174713,0.533719,-0.454779,0.001524,0.030935,10.95,0
3,3,-0.922137,-0.371579,2.132018,-0.796997,0.324175,-1.162006,0.277208,-0.249586,-0.775648,...,-0.302654,-0.246899,-0.045745,0.677110,0.016109,-0.734220,-0.034480,-0.064786,8.04,0
4,4,-1.166525,0.255439,2.108464,0.135019,-0.072979,0.910821,0.755918,0.355528,-0.422820,...,0.136470,0.017496,0.121017,0.636266,0.492943,-0.750242,0.029124,0.091303,175.00,0


In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


In [4]:
from sklearn.utils import resample
df_majority = df[df.Class==0]
df_minority = df[df.Class!=0]
df_minority
 
 #Downsample majority class
df_majority_downsampled = resample(df_majority,replace=False,n_samples=500,random_state=123) 
 

dataset1 = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
dataset1.Class.value_counts()
dataset1

,Index,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
5557,5557,-0.286506,1.483877,1.296656,1.830624,0.804772,-0.620905,1.537253,-0.533306,-1.574283,...,0.035584,0.220838,-0.294050,0.421327,0.292934,-0.017196,-0.300206,-0.210138,20.00,0
189143,189143,0.027315,-2.962887,0.323169,-0.062928,-1.823663,0.883502,-0.427193,0.174294,-0.220772,...,0.455065,0.028891,-0.591603,-0.244121,0.095391,-0.281050,-0.054412,0.121573,657.55,0
217166,217166,1.161457,0.342126,0.596969,1.137035,-0.289921,-0.597790,0.093898,-0.114521,-0.085083,...,0.005647,0.107395,0.038325,0.402180,0.449769,-0.431766,0.043762,0.027593,7.19,0
44582,44582,1.935320,-0.571370,-0.875981,-0.098773,0.061064,0.634513,-0.573312,0.165900,0.777935,...,-0.199225,-0.619102,0.254756,-0.334198,-0.501719,0.278271,-0.050622,-0.045397,64.00,0
152158,152158,1.119941,-0.216024,1.362047,0.959006,-1.276754,-0.491515,-0.592218,0.046830,0.910009,...,-0.058449,-0.021478,0.054220,0.747833,0.202373,0.294721,0.013497,0.033920,28.75,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279108,279108,-1.322789,1.552768,-2.276921,2.992117,-1.947064,-0.480288,-1.362388,0.953242,-2.329629,...,0.614969,-0.195200,0.590711,-0.233378,-0.164285,-0.277498,0.428610,0.246394,270.00,1
279194,279194,-18.474868,11.586381,-21.402917,6.038515,-14.451158,-4.146524,-14.856124,12.431140,-4.053353,...,1.741136,-1.251138,-0.396219,0.095706,1.322751,-0.217955,1.628793,0.482248,99.99,1
279528,279528,0.378275,3.914797,-5.726872,6.094141,1.698875,-2.807314,-0.591118,-0.123496,-2.530713,...,0.149896,-0.601967,-0.613724,-0.403114,1.568445,0.521884,0.527938,0.411910,1.00,1
280854,280854,-1.201398,4.864535,-8.328823,7.652399,-0.167445,-2.767695,-3.176421,1.623279,-4.367228,...,0.532320,-0.556913,0.192444,-0.698588,0.025003,0.514968,0.378105,-0.053133,0.77,1


In [5]:
inputs=dataset1.drop(["Index","Class"],axis="columns")
target=dataset1["Class"]
inputs

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
5557,-0.286506,1.483877,1.296656,1.830624,0.804772,-0.620905,1.537253,-0.533306,-1.574283,0.585061,...,0.020316,0.035584,0.220838,-0.294050,0.421327,0.292934,-0.017196,-0.300206,-0.210138,20.00
189143,0.027315,-2.962887,0.323169,-0.062928,-1.823663,0.883502,-0.427193,0.174294,-0.220772,0.197776,...,1.373300,0.455065,0.028891,-0.591603,-0.244121,0.095391,-0.281050,-0.054412,0.121573,657.55
217166,1.161457,0.342126,0.596969,1.137035,-0.289921,-0.597790,0.093898,-0.114521,-0.085083,-0.071066,...,-0.128026,0.005647,0.107395,0.038325,0.402180,0.449769,-0.431766,0.043762,0.027593,7.19
44582,1.935320,-0.571370,-0.875981,-0.098773,0.061064,0.634513,-0.573312,0.165900,0.777935,0.111132,...,0.046221,-0.199225,-0.619102,0.254756,-0.334198,-0.501719,0.278271,-0.050622,-0.045397,64.00
152158,1.119941,-0.216024,1.362047,0.959006,-1.276754,-0.491515,-0.592218,0.046830,0.910009,-0.249010,...,-0.072864,-0.058449,-0.021478,0.054220,0.747833,0.202373,0.294721,0.013497,0.033920,28.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279108,-1.322789,1.552768,-2.276921,2.992117,-1.947064,-0.480288,-1.362388,0.953242,-2.329629,-3.393553,...,0.988493,0.614969,-0.195200,0.590711,-0.233378,-0.164285,-0.277498,0.428610,0.246394,270.00
279194,-18.474868,11.586381,-21.402917,6.038515,-14.451158,-4.146524,-14.856124,12.431140,-4.053353,-9.040396,...,1.577924,1.741136,-1.251138,-0.396219,0.095706,1.322751,-0.217955,1.628793,0.482248,99.99
279528,0.378275,3.914797,-5.726872,6.094141,1.698875,-2.807314,-0.591118,-0.123496,-2.530713,-5.153095,...,0.440439,0.149896,-0.601967,-0.613724,-0.403114,1.568445,0.521884,0.527938,0.411910,1.00
280854,-1.201398,4.864535,-8.328823,7.652399,-0.167445,-2.767695,-3.176421,1.623279,-4.367228,-5.533443,...,0.597026,0.532320,-0.556913,0.192444,-0.698588,0.025003,0.514968,0.378105,-0.053133,0.77


In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(inputs,target,test_size=0.2)
dataset=pd.concat([X_train,y_train ], axis=1, sort=False)

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
x_train = scaler.fit_transform(X_train) 
x_test = scaler.transform(X_test)

In [8]:
from sklearn.ensemble import GradientBoostingClassifier

In [9]:
model1=GradientBoostingClassifier(n_estimators=1000,max_depth=12,learning_rate=0.2)
#2

In [12]:
model1.fit(x_train,y_train)


GradientBoostingClassifier(learning_rate=0.2, max_depth=12, n_estimators=1000)

RandomForestRegressor(n_estimators=1000, random_state=42)

RandomForestClassifier(n_estimators=200)

In [15]:
model1.score(x_test,y_test)

0.9095744680851063

0.8197185766221814

0.9414893617021277

In [18]:
inputs1=df.drop(["Index","Class"],axis="columns")
target1=df["Class"]
inputs1

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,2.245295,-1.160960,-1.966682,-1.430190,-0.607246,-1.508696,-0.074415,-0.655096,-1.970141,1.607283,...,-0.292081,0.010490,0.521714,-0.064971,0.048849,0.383290,0.103970,-0.047350,-0.064800,74.75
1,1.278609,0.102574,0.512079,0.003930,-0.723474,-1.334105,0.029958,-0.296595,0.128119,-0.209865,...,-0.040015,-0.081470,-0.182506,0.078986,0.789993,0.219794,0.938359,-0.078720,0.008119,1.38
2,1.466457,0.026088,-0.499298,-0.674372,-0.144883,-1.178075,0.058089,-0.420145,-1.359651,0.210249,...,0.207653,-0.234087,-0.710542,-0.001242,-0.174713,0.533719,-0.454779,0.001524,0.030935,10.95
3,-0.922137,-0.371579,2.132018,-0.796997,0.324175,-1.162006,0.277208,-0.249586,-0.775648,-0.061757,...,-0.461062,-0.302654,-0.246899,-0.045745,0.677110,0.016109,-0.734220,-0.034480,-0.064786,8.04
4,-1.166525,0.255439,2.108464,0.135019,-0.072979,0.910821,0.755918,0.355528,-0.422820,-0.842826,...,0.353355,0.136470,0.017496,0.121017,0.636266,0.492943,-0.750242,0.029124,0.091303,175.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284450,1.216221,0.095995,0.666817,1.850616,-0.516269,-0.267296,-0.183114,-0.104976,-1.985790,1.442988,...,-0.528517,-0.286826,-0.347838,0.072935,0.367923,0.358108,0.044019,0.027223,0.028017,19.46
284451,1.429502,-0.990271,-0.224429,-1.808740,0.770788,3.730989,-1.914212,1.036385,-0.316971,0.646103,...,0.206332,0.376136,0.928723,-0.108174,1.048922,0.481214,-0.026342,0.062900,0.024639,0.02
284452,2.041198,-0.218276,-1.405547,0.187570,0.001916,-0.781668,-0.043787,-0.090115,0.677407,0.227002,...,-0.348353,0.273900,0.844092,-0.023770,-0.407890,0.190140,-0.089576,-0.029237,-0.075626,1.00
284453,1.857451,0.071823,-0.086963,3.553877,0.082026,1.245277,-0.788886,0.467363,-0.271086,1.574760,...,-0.306389,0.170106,0.451313,0.157898,0.129330,-0.228721,0.012281,-0.003598,-0.041590,11.31


In [19]:
X1_train,X1_test,y1_train,y1_test=train_test_split(inputs1,target1,test_size=0.4)

In [20]:
x1_train = scaler.fit_transform(X1_train) 
x1_test = scaler.transform(X1_test)

In [24]:
model2=GradientBoostingClassifier(n_estimators=300,max_depth=12,learning_rate=0.2)

In [27]:
model2.fit(x1_train,y1_train)

RandomForestClassifier()

In [ ]:
model2.score(x1_test,y1_test)

0.9995957181276476

In [ ]:
new_test_data = pd.read_csv("TEST.CSV")

In [ ]:
test_new=new_test_data.drop(["Index"],axis="columns")
target1 = model2.predict(test_new)

In [ ]:
res = pd.DataFrame(target1)  
res.index = test_new.index 
res.columns = ["prediction"]
res.to_csv("New_Predicition_Gradient.csv", index = True)  